In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install scikit-image==0.16.2

In [ ]:
import skimage
print(skimage.__version__)

# importing packages

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
from json import dumps
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import tensorflow as tf

# Root directory of the project
ROOT_DIR = os.path.abspath("/../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log



%matplotlib inline 


#####################


MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
#to import bottle.py
!cp /content/gdrive/MyDrive/danaxa\ qualification/samples/bottle/bottle.py /content

In [ ]:
import bottle

## preparing dataset

In [ ]:
config = bottle.BottleConfig()
BOTTLE_DIR = os.path.join(ROOT_DIR, "dataset")
BOTTLE_DATA='/../'

In [ ]:
# Load dataset
# Get the dataset from the releases page
# https://github.com/SajjadParizan/Bottle-Detector
dataset = bottle.BottleDataset()
dataset.load_bottle(BOTTLE_DATA,'train')

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset.image_ids, 10)
for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)

In [ ]:
# Load random image and mask.
image_id = random.choice(dataset.image_ids)
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

## Training

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
# Training dataset.
dataset_train = bottle.BottleDataset()
dataset_train.load_bottle(BOTTLE_DATA, "train")
dataset_train.prepare()

# Validation dataset
dataset_val = bottle.BottleDataset()
dataset_val.load_bottle(BOTTLE_DATA, "val")
dataset_val.prepare()

In [ ]:
model.train(dataset_train, dataset_val,
                learning_rate=0.0001,
                epochs=20,
                layers='heads')

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=0.00001,
            epochs=35, 
            layers="all")

In [ ]:
model_path = os.path.join(MODEL_DIR, "mask_rcnn_bottle2.h5")
model.keras_model.save_weights(model_path)

## Detection

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(MODEL_DIR, "mask_rcnn_bottle2.h5")
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, config, 
                           image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(16, 16))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'])

## Evaluation part

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))